# MLG HW2

In [1]:
import pandas as pd
import numpy as np
import os, time, torch, json
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch_geometric.utils import accuracy,sparse_mx_to_torch_sparse_tensor
import torch_geometric.utils 
# from models.GCN import GCN
import scipy.sparse as sp
from tqdm import tqdm, trange
from torch.autograd import Variable
import torchvision
from torch.utils.data import DataLoader, TensorDataset
# from torchvision import datasets, transforms
# import torch.utils.data as Data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.stats import entropy


import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data, Dataset
# test 
from sklearn.metrics import roc_auc_score, average_precision_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
%autosave 120

/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


Autosaving every 120 seconds


In [2]:
content = []
test_ = []
train_ = []
upload = []
edge_index = [] 
for i in range(3):
    os.chdir('/home/rita/111/111-2MLG/HW2/dataset{}'.format(i + 1))
    temp = pd.read_csv('./content.csv', header = None, sep = '\t')
    temp.sort_values(by = [0], inplace = True)
    temp.set_index([0], inplace = True)
    # temp = torch.Tensor(np.array(temp)).to(torch.float32)
    # print(temp.shape)
    content.append(temp)
    test_.append(pd.read_csv('./test.csv'))
    temp = pd.read_csv('./train.csv')
    # print(temp.shape)
    train_.append(temp)
    temp = temp[temp.label == 1]
    temp = temp[['to', 'from']]
    temp = temp.reset_index(drop = True)
    # print(temp.shape)
    edge_index.append(temp)
    upload.append(pd.read_csv('./upload.csv'))
print('content :\n', content[0][:2])
print('test_ :\n', test_[0].head(2))
print('train_ : \n', train_[0].head(2))
print('edge_index : \n', edge_index[0].head(2))
print('upload : \n', upload[0].head(2))
os.chdir('/home/rita/111/111-2MLG/HW2')

content :
    1     2     3     4     5     6     7     8     9     10    ...  1424  \
0                                                              ...         
0     0     0     0     0     0     0     0     0     0     0  ...     0   
1     0     0     0     0     0     0     0     0     0     0  ...     0   

   1425  1426  1427  1428  1429  1430  1431  1432  1433  
0                                                        
0     0     0     0     0     0     0     0     0     0  
1     1     0     0     0     0     0     0     0     0  

[2 rows x 1433 columns]
test_ :
        id    to  from
0  E10559  2323  2673
1   E4849    81  1634
train_ : 
        id    to  from  label
0  E10311  2399  2339      0
1  E10255  2397  1144      1
edge_index : 
      to  from
0  2397  1144
1  2450  1312
upload : 
        id  prob
0  E10559   0.5
1   E4849   0.5


In [3]:
# content preprocessing
new_features = []
for j in range(len(content)) :
    t = torch.Tensor(np.array(content[j]))
    features_entropy = []
    for i in range(t.shape[1]) :
        temp = t.T[i]
        t1 = torch.sum(temp == 0) / len(temp)
        t2 = torch.sum(temp == 1) / len(temp)
        temp = torch.tensor([t1, t2])
        temp = entropy(temp)
        if (temp == 0) :
            temp = 0
        else :
            temp = 1 / temp
        features_entropy.append(temp)
    features_entropy = torch.tensor(features_entropy).reshape(1, -1)
    t = t * features_entropy
    t = t.type(torch.float32)
    new_features.append(t)
print(new_features[0][:2])
print(content[0].shape)
print(new_features[0].shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
(2708, 1433)
torch.Size([2708, 1433])


In [4]:
# change to Data.Data

# data = {
#     'num_features' : content[i].shape[1], 
#     'num_nodes' : content[i].shape[0], 
#     'x' : content[i], 
#     'edge_index' : torch.Tensor(np.array(train_[i].iloc[::, 1:3])).T.type(torch.long), 
#     'edge_label_index' : torch.Tensor(np.array(train_[i][train_[i].label == 1].iloc[::, 1:3])).T.type(torch.long), 
#     'edge_label' : torch.Tensor(np.array(train_[i][train_[i].label == 1].iloc[::, -1]))
# }

train_pyg = {}
test_pyg = {}
for i in range(3):
    x = torch.Tensor(np.array(content[i]))
    # x = new_features[i]
    train_edge_index = torch.Tensor(np.array(train_[i].iloc[::, 1:3])).T.type(torch.long)
    train_edge_label_index = torch.Tensor(np.array(train_[i][train_[i].label == 1].iloc[::, 1:3])).T.type(torch.long)
    train_edge_label = torch.Tensor(np.array(train_[i][train_[i].label == 1].iloc[::, -1]))
    test_edge_label_index = torch.Tensor(np.array(test_[i].iloc[::, 1:3]).astype(float)).T.type(torch.long)
    train_data = Data(x = x, edge_index = train_edge_index, edge_label_index = train_edge_label_index, edge_label = train_edge_label)
    test_data = Data(x = x, edge_index = train_edge_index, edge_label_index = test_edge_label_index)

    train_pyg[i] = train_data.to(device)
    test_pyg[i] = test_data.to(device)
print(train_pyg[0])
print(test_pyg[0])


Data(x=[2708, 1433], edge_index=[2, 8686], edge_label_index=[2, 4324], edge_label=[4324])
Data(x=[2708, 1433], edge_index=[2, 8686], edge_label_index=[2, 2172])


In [5]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    # T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
    #                   add_negative_train_samples=False),
])
# CUDA_LAUNCH_BLOCKING=1 
for i in range(3) :
    train_pyg[i] = transform(train_pyg[i])
    test_pyg[i] = transform(test_pyg[i])

## Model

In [9]:
class link_predict(nn.Module) :
    def __init__(self, features, emb_dim = 128) :
        super(link_predict, self).__init__()
        self.features = features
        self.edge_index = edge_index
        self.emb_dim = emb_dim
        self.mlp = nn.Sequential(
            nn.Linear(self.features.shape[1], self.emb_dim), 
            nn.ReLU(), 
            nn.Linear(self.emb_dim, self.emb_dim // 2)# , 
            # nn.ReLU(), 
            # nn.Linear(self.emb_dim // 2, self.emb_dim // 4)
        )
        
    def forward(self, want_edge) :
        z = self.mlp(self.features)
        out = []
        for i in range(want_edge.shape[0]) :
            idx1 = want_edge[i, 0].type(torch.LongTensor)
            idx2 = want_edge[i, 1].type(torch.LongTensor)
            temp = (z[idx1] * z[idx2]).sum()
            # temp = torch.matmul(z[idx1], z[idx2].T)
            # temp = temp if temp > 0 else 0
            temp = temp - torch.mean(temp)
            sig = nn.Sigmoid()
            temp = sig(temp)
            out.append(temp)
        return torch.tensor(out).reshape(-1, 1).squeeze()
    
class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels = 128, out_channels = 64):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = nn.functional.leaky_relu(self.conv1(x, edge_index))
        # x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [10]:
def fit(model, optim, loss_fn, train_data):
    model.train()
    optim.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = loss_fn(out, edge_label.to(device))
    loss.backward()
    optim.step()
    return loss

def get_poten_edges(features, edge) :
    poten_edges = []
    
    for i in range(edge.shape[0]) :
        idx1 = edge[i, 0].type(torch.LongTensor)
        idx2 = edge[i, 1].type(torch.LongTensor)
        temp = (features[idx1] - features[idx2]) ** 2
        temp = torch.sum((features[idx1] - features[idx2]) ** 2)
        # print(temp)
        # temp = (features[idx1] == features[idx2]).sum() 
        poten_edges.append(temp)
    temp -= temp.min()        
    return torch.tensor(poten_edges)   

def training_loop(data, train_data, n_epochs):
    model1 = Net(in_channels = data.x.shape[1]).to(device)
    optim1 = torch.optim.Adam(params=model1.parameters(), lr=1e-2)
    loss_fn1 = torch.nn.BCEWithLogitsLoss()

    model2 = link_predict(data.x).to(device)
    optim2 = torch.optim.Adam(model2.parameters(), lr=1e-4, weight_decay = 1e-5)  
    loss_fn2 = nn.MSELoss()

    features = data.x.to(torch.float32)
    train_x = torch.Tensor(np.array(train_data.iloc[::, [1, 2]])).to(torch.float32)
    train_y = torch.Tensor(np.array(train_data.iloc[::, -1])).to(torch.float32)
    # train_dataset = Data.TensorDataset(train_x, train_y)
    # train_loader = DataLoader(
    #     dataset = train_dataset,
    #     batch_size = 64,
    #     shuffle = True,
    #     num_workers = 4
    # )
    edge = train_x.to(device)
    labels = train_y.to(device)

    for epoch in trange(1, n_epochs + 1):
        # pyg
        model1.train()
        optim1.zero_grad()
        z = model1.encode(data.x, data.edge_index)

        # We perform a new round of negative sampling for every training epoch:
        neg_edge_index = negative_sampling(
            edge_index=data.edge_index, num_nodes=data.num_nodes,
            num_neg_samples=data.edge_label_index.size(1), method='sparse')
        edge_label_index = torch.cat(
            [data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            data.edge_label,
            data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model1.decode(z, edge_label_index).view(-1)
        loss1 = loss_fn1(out, edge_label.to(device))

        # self
        poten = get_poten_edges(features, edge).to(device)
        outputs = model2(edge).to(device)            
        loss2 = 0
        for j in range(labels.shape[0]) :
            if(labels[j] == 1):
                loss2 += torch.exp(-(poten[j]/100**2)) * loss_fn2(outputs[j], labels[j])
            else :
                loss2 += torch.exp((poten[j]/100**2)) * loss_fn2(outputs[j], labels[j])     
        loss2 /= edge.shape[0]
        loss = loss1 + loss2

        loss.requires_grad_()
        optim2.zero_grad() 
        loss.backward() 
        optim1.step()
        optim2.step()
        # loop.set_description(f'Epoch[{epoch} / {n_epochs}]')

        if epoch % 250 == 0 :
            print('Epoch : {}, Loss : {}'.format(epoch, loss))
    
    return model1, model2

In [11]:
s = time.time()
for i in range(1, 2):
    # if i == 2 :
    #     n_epochs = 2000
    # else :
    #     n_epochs = 500
    n_epochs = 500
    model1, model2 = training_loop(data = train_pyg[i], train_data = train_[i], n_epochs = n_epochs)
    # pred = predict(model, test_pyg[i])

    z = model1.encode(test_pyg[i].x, test_pyg[i].edge_index)
    pred1 = torch.sigmoid(model1.decode(z, test_pyg[i].edge_label_index)).cpu().detach().numpy().reshape(-1, 1)
    # print(type(pred1))
    # print(pred1.shape)
    # pred1 = pd.DataFrame(pred1)
    # print(pred1.shape)
    z = model2(test_pyg[i].edge_label_index.T).cpu().detach().numpy().reshape(-1, 1)
    # print(type(z))
    # print(z.shape)
    # z = pd.DataFrame(z)
    # print(z.shape)
    pred = np.hstack([pred1, z])
    pred = np.mean(pred, axis = 1)
    # print(pred.shape)
    
    pred = pd.DataFrame(pred)

    # pred = pd.DataFrame(pred.cpu().detach().numpy())
    pred = pd.concat([test_[i], pred], axis = 1)
    pred = pred.drop(['to', 'from'], axis = 1)
    pred.columns = ['id', 'prob']
    # print(pred.shape)
    pred.to_csv('./upload/pred_pyg_mix_norm_500_{}.csv'.format(i + 1))

print(time.time() - s)

 50%|█████     | 250/500 [1:25:21<1:13:50, 17.72s/it]

Epoch : 250, Loss : 0.6648212671279907


100%|██████████| 500/500 [2:21:55<00:00, 17.03s/it]  

Epoch : 500, Loss : 0.6473313570022583


8516.733285665512


In [ ]:
def predict(test_data, model):
    z = model.encode(test_data.x, test_data.edge_index)
    test_pred = torch.sigmoid(model.decode(z, test_data.edge_label_index))
    return test_pred